In [106]:
import os
import time
import requests
import pandas as pd
import numpy as np
from IPython.display import display

AV_API_KEY = "ZLVG8A91U2PYH4H2"
BASE = "https://www.alphavantage.co/query"

def fetch_alpha_vantage(function_name: str, symbol: str, start_year=2020, end_year=2024) -> pd.DataFrame:
    """
    function_name: "BALANCE_SHEET" | "INCOME_STATEMENT" | "CASH_FLOW"
    Alpha Vantage back to annualReports / quarterlyReports
    """
    if not AV_API_KEY:
        raise RuntimeError("No detction of ALPHAVANTAGE_API_KEY. Please export ALPHAVANTAGE_API_KEY='你的key'")

    params = {
        "function": function_name,
        "symbol": symbol,
        "apikey": AV_API_KEY,
    }

    r = requests.get(BASE, params=params, timeout=30)
    r.raise_for_status()
    data = r.json()

    reports = data.get("annualReports", [])
    df = pd.DataFrame(reports)

    if "fiscalDateEnding" in df.columns:
        df["fiscalDateEnding"] = pd.to_datetime(df["fiscalDateEnding"], errors="coerce")
        df = df[df["fiscalDateEnding"].dt.year.between(start_year, end_year)]
        df = df.sort_values("fiscalDateEnding", ascending=True)

    return df

def save_outputs(df: pd.DataFrame, prefix: str):
    df.to_csv(f"{prefix}.csv", index=False, encoding="utf-8-sig")
    df.to_excel(f"{prefix}.xlsx", index=False)

def run(symbol="BP"):
    bs = fetch_alpha_vantage("BALANCE_SHEET", symbol)
    is_ = fetch_alpha_vantage("INCOME_STATEMENT", symbol)
    cf = fetch_alpha_vantage("CASH_FLOW", symbol)

    save_outputs(bs, f"{symbol}_balance_sheet_2020_2024")
    save_outputs(is_, f"{symbol}_income_statement_2020_2024")
    save_outputs(cf, f"{symbol}_cash_flow_2020_2024")

    print("Finished：CSV/XLSX files have been exported.")

if __name__ == "__main__":
    run(symbol="BP")

Finished：CSV/XLSX files have been exported.


In [107]:
import requests, pandas as pd, time
from IPython.display import display

AV_API_KEY = "Paste your key here."
BASE = "https://www.alphavantage.co/query"

def fetch_raw(function_name: str, symbol: str):
    print(f"\n=== Initiate Request: {function_name} / {symbol} ===")
    params = {"function": function_name, "symbol": symbol, "apikey": AV_API_KEY}

    try:
        r = requests.get(BASE, params=params, timeout=30)
        print("HTTP Status Codes:", r.status_code)
        r.raise_for_status()
        text_preview = r.text[:300].replace("\n", " ")
        print("Response Preview (First 300 Characters):", text_preview)

        data = r.json()
        print("Return to top-level keys:", list(data.keys())[:10])
        return data
    except Exception as e:
        print("Request/Parsing Failed:", repr(e))
        return None

def to_annual_df(data):
    if not data:
        return None
    if "Error Message" in data:
        print("Interface error(Error Message):", data["Error Message"])
        return None
    if "Note" in data:
        print("Trigger Frequency Limit(Note):", data["Note"])
        return None

    reports = data.get("annualReports")
    if not reports:
        print("The annualReports field is either missing or empty.")
        return None

    df = pd.DataFrame(reports)
    if "fiscalDateEnding" in df.columns:
        df["fiscalDateEnding"] = pd.to_datetime(df["fiscalDateEnding"], errors="coerce")
        df = df[df["fiscalDateEnding"].dt.year.between(2020, 2024)].sort_values("fiscalDateEnding")
    return df

symbol = "BP"

data_bs = fetch_raw("BALANCE_SHEET", symbol)
time.sleep(15)

data_is = fetch_raw("INCOME_STATEMENT", symbol)
time.sleep(15)

data_cf = fetch_raw("CASH_FLOW", symbol)

bs = to_annual_df(data_bs)
is_ = to_annual_df(data_is)
cf = to_annual_df(data_cf)

print("\n=== Begin displaying the DataFrame ===")
print("Balance Sheet df:", None if bs is None else bs.shape)
print("Income Statement df:", None if is_ is None else is_.shape)
print("Cash Flow Statement df:", None if cf is None else cf.shape)

if bs is not None:
    print("\n📘 Balance Sheet")
    display(bs)
if is_ is not None:
    print("\n📗 Income Statement")
    display(is_)
if cf is not None:
    print("\n📙 Cash Flow Statement")
    display(cf)

print("\n=== End of script: If you can see this line, it means the cell's output mechanism is working fine. ===")



=== Initiate Request: BALANCE_SHEET / BP ===
HTTP Status Codes: 200
Response Preview (First 300 Characters): {     "symbol": "BP",     "annualReports": [         {             "fiscalDateEnding": "2024-12-31",             "reportedCurrency": "USD",             "totalAssets": "282228000000",             "totalCurrentAssets": "102834000000",             "cashAndCashEquivalentsAtCarryingValue": "34360000000",
Return to top-level keys: ['symbol', 'annualReports', 'quarterlyReports']

=== Initiate Request: INCOME_STATEMENT / BP ===
HTTP Status Codes: 200
Response Preview (First 300 Characters): {     "symbol": "BP",     "annualReports": [         {             "fiscalDateEnding": "2024-12-31",             "reportedCurrency": "USD",             "grossProfit": "30239000000",             "totalRevenue": "189185000000",             "costOfRevenue": "158946000000",             "costofGoodsAndSe
Return to top-level keys: ['symbol', 'annualReports', 'quarterlyReports']

=== Initiate Request: CASH

,fiscalDateEnding,reportedCurrency,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,...,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding
4,2020-12-31,USD,267654000000,72982000000,29194000000,29194000000,16873000000,17849000000,194672000000,114836000000,...,9359000000,None,81926000000,22209000000,None,71250000000,-13224000000,47300000000,5362000000,3370251999
3,2021-12-31,USD,287272000000,92590000000,25941000000,25941000000,23711000000,26363000000,194682000000,112902000000,...,5557000000,None,69787000000,31394000000,None,75463000000,-12624000000,51815000000,5194000000,3376730999
2,2022-12-31,USD,288120000000,107688000000,23329000000,23329000000,28081000000,33288000000,180432000000,106044000000,...,3198000000,None,55493000000,40082000000,None,67553000000,None,34732000000,4774000000,3164655999
1,2023-12-31,USD,280294000000,104146000000,27748000000,27748000000,22819000000,30335000000,176148000000,104719000000,...,3284000000,None,63075000000,32742000000,None,70283000000,None,35339000000,4475000000,2958345999
0,2024-12-31,USD,282228000000,102834000000,34360000000,34360000000,23232000000,26185000000,179394000000,100238000000,...,4474000000,None,71547000000,32777000000,None,59246000000,None,22531000000,4165000000,2802776999



📗 Income Statement


,fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,...,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome
4,2020-12-31,USD,10184000000,105944000000,95760000000,95760000000,-573000000,10397000000,332000000,10757000000,...,None,14889000000,-24888000000,-4159000000,None,-20729000000,None,-21931000000,-7042000000,-20305000000
3,2021-12-31,USD,22860000000,157739000000,134879000000,134879000000,10672000000,11931000000,266000000,12188000000,...,None,14805000000,15227000000,6740000000,None,8487000000,None,17744000000,32549000000,7565000000
2,2022-12-31,USD,55096000000,241392000000,186296000000,186296000000,41447000000,13449000000,274000000,13649000000,...,None,14318000000,15405000000,16762000000,None,-1357000000,None,18028000000,32346000000,-2487000000
1,2023-12-31,USD,64057000000,208351000000,144294000000,144294000000,46977000000,16772000000,298000000,17080000000,...,None,15928000000,23749000000,7869000000,None,26605000000,None,27575000000,43503000000,15239000000
0,2024-12-31,USD,30239000000,189185000000,158946000000,158946000000,13615000000,16417000000,301000000,16624000000,...,None,16622000000,6782000000,5553000000,None,1229000000,None,11361000000,27983000000,381000000



📙 Cash Flow Statement


,fiscalDateEnding,reportedCurrency,operatingCashflow,paymentsForOperatingActivities,proceedsFromOperatingActivities,changeInOperatingLiabilities,changeInOperatingAssets,depreciationDepletionAndAmortization,capitalExpenditures,changeInReceivables,...,dividendPayoutCommonStock,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome
4,2020-12-31,USD,12162000000,None,None,None,None,14889000000,12306000000,None,...,6340000000,None,None,None,None,-776000000,None,None,None,-24888000000
3,2021-12-31,USD,23612000000,None,None,None,None,14972000000,10887000000,None,...,4304000000,None,None,None,None,-3151000000,None,None,None,15227000000
2,2022-12-31,USD,40932000000,None,None,None,None,14703000000,12069000000,None,...,4358000000,None,None,None,None,-9996000000,None,None,None,15405000000
1,2023-12-31,USD,32039000000,None,None,None,None,15928000000,14285000000,None,...,4809000000,None,None,None,None,-7918000000,None,None,None,23749000000
0,2024-12-31,USD,27297000000,None,None,None,None,16622000000,15297000000,None,...,5003000000,None,None,None,None,-7127000000,None,None,None,6782000000



=== End of script: If you can see this line, it means the cell's output mechanism is working fine. ===


In [108]:
def clean_financial_df(df, name="Unknown"):
    """
    Robust cleaner:
    - Accepts None
    - Accepts empty df
    - Converts fiscalDateEnding to index
    - Converts numeric columns safely
    """

    if df is None:
        print(f"❌ {name}: Input is None (Possible API rate limiting)")
        return None

    if not isinstance(df, pd.DataFrame):
        print(f"❌ {name}: The input is not a DataFrame.")
        return None

    if df.empty:
        print(f"❌ {name}: The DataFrame is empty.")
        return None

    df = df.copy()

    if "fiscalDateEnding" not in df.columns:
        print(f"❌ {name}: The fiscalDateEnding column is missing.")
        return None

    df["fiscalDateEnding"] = pd.to_datetime(df["fiscalDateEnding"], errors="coerce")
    df = df.dropna(subset=["fiscalDateEnding"])
    df = df.set_index("fiscalDateEnding").sort_index()

    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    print(f"✅ {name}: Cleaning finished，shape = {df.shape}")
    return df

In [109]:
bs_clean = clean_financial_df(bs, "Balance Sheet")
is_clean = clean_financial_df(is_, "Income Statement")
cf_clean = clean_financial_df(cf, "Cash Flow")

✅ Balance Sheet: Cleaning finished，shape = (5, 37)
✅ Income Statement: Cleaning finished，shape = (5, 25)
✅ Cash Flow: Cleaning finished，shape = (5, 28)


In [153]:
def validate_financials_final(bs, is_, cf, start_year=2020, end_year=2024):
    issues = []

    if bs is None:
        issues.append("[Existence] Balance Sheet not available ")
    if is_ is None:
        issues.append("[Existence] Income Statement not available ")
    if cf is None:
        issues.append("[Existence] Cash Flow not available ")
    if issues:
        return issues

    expected_years = set(range(start_year, end_year + 1))
    for df, name in [(bs, "Balance Sheet"), (is_, "Income Statement"), (cf, "Cash Flow")]:
        years = set(df.index.year)
        missing = expected_years - years
        if missing:
            issues.append(f"[Completeness] {name} Missing years: {sorted(missing)}")

    required_fields = {
        "Cash Flow": ["operatingCashflow", "capitalExpenditures"],
        "Income Statement": ["netIncome"],
        "Balance Sheet": ["totalAssets", "totalLiabilities"]
    }
    for name, fields in required_fields.items():
        df = {"Cash Flow": cf, "Income Statement": is_, "Balance Sheet": bs}[name]
        for field in fields:
            if field not in df.columns:
                issues.append(f"[Readiness] {name} Missing field: {field}")

    if "totalAssets" in bs.columns and "totalLiabilities" in bs.columns:
        bs["impliedEquity"] = bs["totalAssets"] - bs["totalLiabilities"]
    else:
        issues.append("[Accounting] The absence of totalAssets or totalLiabilities prevents the generation of impliedEquity.")

    shares_candidates = [
        "weightedAverageShsOutDiluted",
        "weightedAverageShsOut",
        "commonStockSharesOutstanding"
    ]
    shares_ok = any(field in is_.columns for field in shares_candidates) or any(field in bs.columns for field in shares_candidates)
    if not shares_ok:
        issues.append("[Shares] Shares Outstanding Missing: Unable to calculate EPS/P/E")

    return issues

In [111]:
issues = validate_financials_final(bs_clean, is_clean, cf_clean)

if not issues:
    print("✅ Data validation passed: Proceed with DCF/Multiple analysis.")
else:
    print("⚠️ Data validation identified issues：")
    for i in issues:
        print("-", i)

✅ Data validation passed: Proceed with DCF/Multiple analysis.


In [112]:
pd.set_option("display.float_format", "{:.6f}".format)

def compute_ratios(bs, is_, cf):
   
    def safe_get(df, col):
        return df[col] if col in df.columns else None


    net_income = safe_get(is_, "netIncome")
    total_assets = safe_get(bs, "totalAssets")
    equity = safe_get(bs, "impliedEquity")  
    revenue = safe_get(is_, "totalRevenue")
    ebitda = safe_get(is_, "ebitda")

    ratios = pd.DataFrame(index=bs.index)

    if net_income is not None and equity is not None:
        ratios["ROE"] = net_income / equity

    if net_income is not None and total_assets is not None:
        ratios["ROA"] = net_income / total_assets

    if net_income is not None and revenue is not None:
        ratios["Net_Margin"] = net_income / revenue

    if ebitda is not None and revenue is not None:
        ratios["EBITDA_Margin"] = ebitda / revenue

    total_liabilities = safe_get(bs, "totalLiabilities")
    if total_liabilities is not None and equity is not None:
        ratios["Debt_to_Equity"] = total_liabilities / equity

    current_assets = safe_get(bs, "totalCurrentAssets")
    current_liabilities = safe_get(bs, "totalCurrentLiabilities")
    if current_assets is not None and current_liabilities is not None:
        ratios["Current_Ratio"] = current_assets / current_liabilities

    interest_expense = safe_get(is_, "interestExpense")
    ebit = safe_get(is_, "ebit")
    if ebit is not None and interest_expense is not None:
        ratios["Interest_Coverage"] = ebit / (-interest_expense)

    if revenue is not None and total_assets is not None:
        ratios["Asset_Turnover"] = revenue / total_assets

    inventory = safe_get(bs, "inventory")
    cogs = safe_get(is_, "costofGoodsAndServicesSold")
    if inventory is not None and cogs is not None:
        ratios["Inventory_Turnover"] = cogs / inventory

    op_cf = safe_get(cf, "operatingCashflow")
    capex = safe_get(cf, "capitalExpenditures")
    if op_cf is not None and capex is not None:
        ratios["FCF"] = op_cf - capex
        ratios["FCF_Margin"] = ratios["FCF"] / revenue

    return ratios

ratio_df = compute_ratios(bs_clean, is_clean, cf_clean)

ratio_df.round(6)

,ROE,ROA,Net_Margin,EBITDA_Margin,Debt_to_Equity,Current_Ratio,Interest_Coverage,Asset_Turnover,Inventory_Turnover,FCF,FCF_Margin
fiscalDateEnding,,,,,,,,,,,
2020-12-31,-0.237297,-0.075863,-0.191658,-0.066469,2.127968,1.220455,10.162651,0.395824,5.675339,-144000000,-0.001359
2021-12-31,0.083648,0.026334,0.047959,0.206347,2.176417,1.153238,-7.049662,0.549093,5.688457,12725000000,0.080671
2022-12-31,-0.029967,-0.008632,-0.010303,0.133998,2.471744,1.087560,-6.873046,0.837818,6.634237,28863000000,0.119569
2023-12-31,0.178249,0.054368,0.073141,0.208797,2.278561,1.209903,-9.414476,0.743330,6.323415,17754000000,0.085212
2024-12-31,0.004865,0.001350,0.002014,0.147913,2.603616,1.250398,-2.481109,0.670327,6.841684,12000000000,0.063430


In [113]:
def compute_historical_fcf(cf):
    cf = cf.copy()
    cf["operatingCashflow"] = pd.to_numeric(cf["operatingCashflow"], errors="coerce")
    cf["capitalExpenditures"] = pd.to_numeric(cf["capitalExpenditures"], errors="coerce")
    cf["FCF"] = cf["operatingCashflow"] - cf["capitalExpenditures"]
    return cf[["FCF"]]

hist_fcf = compute_historical_fcf(cf_clean)
hist_fcf

,FCF
fiscalDateEnding,
2020-12-31,-144000000
2021-12-31,12725000000
2022-12-31,28863000000
2023-12-31,17754000000
2024-12-31,12000000000


In [114]:
import numpy as np

def forecast_fcf_4yr_cagr(hist_fcf, years=5):
    hist_fcf = hist_fcf.dropna()

    recent = hist_fcf.tail(4)

    if len(recent) < 2:
        last_value = hist_fcf["FCF"].iloc[-1]
        future_index = pd.date_range(
            start=hist_fcf.index[-1] + pd.DateOffset(years=1),
            periods=years,
            freq="YE"
        )
        return pd.Series([last_value]*years, index=future_index, name="FCF"), 0.0

    start = recent["FCF"].iloc[0]
    end = recent["FCF"].iloc[-1]
    n = len(recent) - 1

    if start <= 0 or end <= 0:
        cagr = 0.0
    else:
        cagr = (end / start) ** (1/n) - 1

    last_value = recent["FCF"].iloc[-1]
    forecast = []
    for i in range(1, years + 1):
        last_value = last_value * (1 + cagr)
        forecast.append(last_value)

    future_index = pd.date_range(
        start=hist_fcf.index[-1] + pd.DateOffset(years=1),
        periods=years,
        freq="YE"
    )
    fcf_forecast = pd.Series(forecast, index=future_index, name="FCF")
    return fcf_forecast, cagr

fcf_forecast, cagr = forecast_fcf_4yr_cagr(hist_fcf, years=5)

print("CAGR over the past 4 years:", cagr)
fcf_forecast

CAGR over the past 4 years: -0.019364032028542977


2025-12-31   11767631615.657484
2026-12-31   11539762820.151798
2027-12-31   11316306483.300589
2028-12-31   11097177162.113148
2029-12-31   10882291068.119574
Freq: YE-DEC, Name: FCF, dtype: float64

In [115]:
import requests
import pandas as pd

API_KEY = "Your Alpha Vantage API Key"

def fetch_overview(symbol: str) -> dict:
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "OVERVIEW",
        "symbol": symbol,
        "apikey": API_KEY
    }
    r = requests.get(url, params=params)
    data = r.json()
    return data

In [44]:
overview = fetch_overview("BP")
overview

{'Symbol': 'BP',
 'AssetType': 'Common Stock',
 'Name': 'BP PLC ADR',
 'Description': 'BP PLC is a prominent global energy corporation based in London, engaged in the exploration, production, refining, and distribution of oil and natural gas. The company is proactively transitioning towards a low-carbon future by integrating renewable energy initiatives and innovative technologies into its operations, reflecting its commitment to sustainability and alignment with international climate objectives. With a diverse portfolio and extensive geographic footprint, BP is well-positioned to navigate evolving energy market dynamics while maintaining a strong focus on environmental stewardship and corporate governance.',
 'CIK': '313807',
 'Exchange': 'NYSE',
 'Currency': 'USD',
 'Country': 'USA',
 'Sector': 'ENERGY',
 'Industry': 'OIL & GAS INTEGRATED',
 'Address': "1 ST JAMES'S SQUARE, LONDON, UNITED KINGDOM, SW1Y 4PD",
 'OfficialSite': 'https://www.bp.com',
 'FiscalYearEnd': 'December',
 'Lates

In [154]:
def estimate_wacc(bs, is_, overview, risk_free_rate=0.04, market_risk_premium=0.06):
   
    if "MarketCapitalization" not in overview:
        raise ValueError("Unable to retrieve market cap. Please check the Overview interface.")
    market_cap = float(overview["MarketCapitalization"])

    if "totalDebt" in bs.columns:
        total_debt = float(bs["totalDebt"].iloc[-1])
    else:
        lt = float(bs["longTermDebt"].iloc[-1]) if "longTermDebt" in bs.columns else 0
        st = float(bs["shortTermDebt"].iloc[-1]) if "shortTermDebt" in bs.columns else 0
        total_debt = lt + st

    cash = float(bs["cashAndCashEquivalentsAtCarryingValue"].iloc[-1])
    net_debt = total_debt - cash

    ev = market_cap + net_debt
    w_e = market_cap / ev
    w_d = net_debt / ev

    beta_raw = float(overview.get("Beta", np.nan))

    beta_min, beta_max = 0.6, 1.2

    if np.isnan(beta_raw) or beta_raw < beta_min or beta_raw > beta_max:
        beta = 0.9  
        beta_used = "Industry median (0.9) replacement"
    else:
        beta = beta_raw
        beta_used = f"API beta({beta_raw})"

    cost_of_equity = risk_free_rate + beta * market_risk_premium

    interest_expense = abs(float(is_["interestExpense"].iloc[-1])) if "interestExpense" in is_.columns else 0
    cost_of_debt = interest_expense / total_debt if total_debt != 0 else 0.05

    tax_rate = 0.25
    wacc = w_e * cost_of_equity + w_d * cost_of_debt * (1 - tax_rate)

    return {
        "market_cap": market_cap,
        "total_debt": total_debt,
        "cash": cash,
        "net_debt": net_debt,
        "ev": ev,
        "w_e": w_e,
        "w_d": w_d,
        "beta_raw": beta_raw,
        "beta": beta,
        "beta_used": beta_used,
        "cost_of_equity": cost_of_equity,
        "cost_of_debt": cost_of_debt,
        "tax_rate": tax_rate,
        "wacc": wacc
    }

wacc_info = estimate_wacc(bs_clean, is_clean, overview)
wacc_info

{'market_cap': 91322286000.0,
 'total_debt': 62207000000.0,
 'cash': 34360000000.0,
 'net_debt': 27847000000.0,
 'ev': 119169286000.0,
 'w_e': 0.7663240174150242,
 'w_d': 0.2336759825849758,
 'beta_raw': 0.028,
 'beta': 0.9,
 'beta_used': 'Industry median (0.9) replacement',
 'cost_of_equity': 0.094,
 'cost_of_debt': 0.07360907936405871,
 'tax_rate': 0.25,
 'wacc': 0.08493496309769119}

In [117]:
forecast_years = 5            
terminal_growth_rate = 0.02   
wacc = wacc_info["wacc"]      

In [118]:
def forecast_fcf(hist_fcf, years=5, growth_rate=None):
    last_fcf = hist_fcf["FCF"].iloc[-1]

    if growth_rate is None:
        yoy = hist_fcf["FCF"].pct_change().dropna()
        growth_rate = yoy.mean()

    future = []
    fcf = last_fcf
    for _ in range(years):
        fcf = fcf * (1 + growth_rate)
        future.append(fcf)

    future_index = pd.date_range(
        start=hist_fcf.index[-1] + pd.DateOffset(years=1),
        periods=years,
        freq="YE"   
    )

    return pd.Series(future, index=future_index, name="FCF")

future_fcf = forecast_fcf(hist_fcf, years=forecast_years, growth_rate=fcf_growth_rate)
future_fcf

2025-12-31   11767631615.657484
2026-12-31   11539762820.151798
2027-12-31   11316306483.300589
2028-12-31   11097177162.113148
2029-12-31   10882291068.119574
Freq: YE-DEC, Name: FCF, dtype: float64

In [119]:
def terminal_value(last_fcf, wacc, terminal_growth):
    return last_fcf * (1 + terminal_growth) / (wacc - terminal_growth)

tv = terminal_value(future_fcf.iloc[-1], wacc, terminal_growth_rate)

tv_pv = tv / ((1 + wacc) ** forecast_years)
tv, tv_pv

(np.float64(170939296181.36078), np.float64(113716474376.60283))

In [122]:
def discount_cashflows(cashflows, discount_rate):
    years = range(1, len(cashflows) + 1)
    discount_factors = [(1 + discount_rate) ** t for t in years]
    discounted = cashflows.values / discount_factors
    return pd.Series(discounted, index=cashflows.index)

discounted_fcf = discount_cashflows(future_fcf, wacc)
discounted_fcf

2025-12-31   10846393577.416573
2026-12-31    9803687803.018623
2027-12-31    8861221368.471533
2028-12-31    8009357878.254676
2029-12-31    7239387320.827696
Freq: YE-DEC, dtype: float64

In [124]:
ev_dcf = discounted_fcf.sum() + tv_pv

net_debt = wacc_info["net_debt"]
equity_value = ev_dcf - net_debt

shares_outstanding = float(overview.get("SharesOutstanding", 1))
price_per_share = equity_value / shares_outstanding

ev_dcf, equity_value, price_per_share

(np.float64(158476522324.59192),
 np.float64(130629522324.59192),
 np.float64(50.84182282577414))

In [136]:
import yfinance as yf

peers = {
    "BP": "BP",
    "Shell": "SHEL",
    "ExxonMobil": "XOM"
}

In [137]:
def fetch_multiples_extended(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info

    market_cap = info.get("marketCap", None)
    net_income = info.get("netIncomeToCommon", None)
    operating_cf = info.get("operatingCashflow", None)
    ebitda = info.get("ebitda", None)

    total_debt = info.get("totalDebt", None)
    cash = info.get("totalCash", None)

    if market_cap is not None and total_debt is not None and cash is not None:
        ev = market_cap + total_debt - cash
    else:
        ev = None

    pe = market_cap / net_income if market_cap and net_income and net_income > 0 else np.nan
    ev_ebitda = ev / ebitda if ev and ebitda and ebitda > 0 else np.nan
    ev_ocf = ev / operating_cf if ev and operating_cf and operating_cf > 0 else np.nan

    return {
        "Market Cap": market_cap,
        "EV": ev,
        "Net Income (TTM)": net_income,
        "EBITDA (TTM)": ebitda,
        "Operating CF (TTM)": operating_cf,
        "P/E": pe,
        "EV/EBITDA": ev_ebitda,
        "EV/Operating CF": ev_ocf
    }

In [139]:
rows = []

for name, ticker in peers.items():
    data = fetch_multiples_extended(ticker)
    data["Company"] = name
    rows.append(data)

df_multiples = pd.DataFrame(rows).set_index("Company")
df_multiples

,Market Cap,EV,Net Income (TTM),EBITDA (TTM),Operating CF (TTM),P/E,EV/EBITDA,EV/Operating CF
Company,,,,,,,,
BP,91064860672,130833862656,1516999936,28657999872,24317999104,60.029574,4.565352,5.380124
Shell,208867262464,249791266816,14631000064,48754999296,46590001152,14.275665,5.123398,5.361478
ExxonMobil,560020127744,588242126848,29952999424,61694001152,51520000000,18.696629,9.534835,11.417743


In [140]:
multiples_table = df_multiples[[
    "EV",
    "EBITDA (TTM)",
    "EV/EBITDA"
]]

multiples_table

,EV,EBITDA (TTM),EV/EBITDA
Company,,,
BP,130833862656,28657999872,4.565352
Shell,249791266816,48754999296,5.123398
ExxonMobil,588242126848,61694001152,9.534835


In [141]:
multiples_core = df_multiples[[
    "P/E",
    "EV/EBITDA",
    "EV/Operating CF"
]]

multiples_core


,P/E,EV/EBITDA,EV/Operating CF
Company,,,
BP,60.029574,4.565352,5.380124
Shell,14.275665,5.123398,5.361478
ExxonMobil,18.696629,9.534835,11.417743


In [142]:
summary = multiples_core.agg(["mean", "median", "min", "max"])
summary

,P/E,EV/EBITDA,EV/Operating CF
mean,31.000623,6.407862,7.386449
median,18.696629,5.123398,5.380124
min,14.275665,4.565352,5.361478
max,60.029574,9.534835,11.417743


In [143]:
bp_data = df_multiples.loc["BP"]

bp_net_income = bp_data["Net Income (TTM)"]
bp_ebitda = bp_data["EBITDA (TTM)"]
bp_ocf = bp_data["Operating CF (TTM)"]
bp_net_debt = bp_data["EV"] - bp_data["Market Cap"]

bp_data

Market Cap            91064860672.000000
EV                   130833862656.000000
Net Income (TTM)       1516999936.000000
EBITDA (TTM)          28657999872.000000
Operating CF (TTM)    24317999104.000000
P/E                            60.029574
EV/EBITDA                       4.565352
EV/Operating CF                 5.380124
Name: BP, dtype: float64

In [144]:
industry_median = multiples_core.median()
industry_median

P/E               18.696629
EV/EBITDA          5.123398
EV/Operating CF    5.380124
dtype: float64

In [145]:
valuation_rows = []

if not np.isnan(industry_median["P/E"]) and bp_net_income > 0:
    implied_equity_pe = industry_median["P/E"] * bp_net_income
    valuation_rows.append({
        "Method": "P/E",
        "Implied Enterprise Value": np.nan,
        "Implied Equity Value": implied_equity_pe
    })

if not np.isnan(industry_median["EV/EBITDA"]) and bp_ebitda > 0:
    implied_ev_ebitda = industry_median["EV/EBITDA"] * bp_ebitda
    implied_equity_ebitda = implied_ev_ebitda - bp_net_debt
    valuation_rows.append({
        "Method": "EV/EBITDA",
        "Implied Enterprise Value": implied_ev_ebitda,
        "Implied Equity Value": implied_equity_ebitda
    })

if not np.isnan(industry_median["EV/Operating CF"]) and bp_ocf > 0:
    implied_ev_ocf = industry_median["EV/Operating CF"] * bp_ocf
    implied_equity_ocf = implied_ev_ocf - bp_net_debt
    valuation_rows.append({
        "Method": "EV/Operating CF",
        "Implied Enterprise Value": implied_ev_ocf,
        "Implied Equity Value": implied_equity_ocf
    })

df_implied_value = pd.DataFrame(valuation_rows)
df_implied_value

,Method,Implied Enterprise Value,Implied Equity Value
0,P/E,NaN,28362785506.738033
1,EV/EBITDA,146826339776.543701,107057337792.543701
2,EV/Operating CF,130833862656.000000,91064860672.000000


In [146]:
shares_outstanding = yf.Ticker("BP").info.get("sharesOutstanding")

df_implied_value["Implied Share Price"] = (
    df_implied_value["Implied Equity Value"] / shares_outstanding
)

df_implied_value

,Method,Implied Enterprise Value,Implied Equity Value,Implied Share Price
0,P/E,NaN,28362785506.738033,11.041150
1,EV/EBITDA,146826339776.543701,107057337792.543701,41.675601
2,EV/Operating CF,130833862656.000000,91064860672.000000,35.450002


In [147]:
dcf_share_price = price_per_share

In [148]:
valuation_summary = df_implied_value[[
    "Method",
    "Implied Equity Value",
    "Implied Share Price"
]].copy()

valuation_summary = pd.concat([
    valuation_summary,
    pd.DataFrame([{
        "Method": "DCF",
        "Implied Equity Value": equity_value,
        "Implied Share Price": dcf_share_price
    }])
], ignore_index=True)

valuation_summary

,Method,Implied Equity Value,Implied Share Price
0,P/E,28362785506.738033,11.041150
1,EV/EBITDA,107057337792.543701,41.675601
2,EV/Operating CF,91064860672.000000,35.450002
3,DCF,130629522324.591919,50.841823


In [149]:
valuation_summary_formatted = valuation_summary.copy()

valuation_summary_formatted["Implied Equity Value (USD bn)"] = (
    valuation_summary_formatted["Implied Equity Value"] / 1e9
).round(2)

valuation_summary_formatted["Implied Share Price (USD)"] = (
    valuation_summary_formatted["Implied Share Price"]
).round(2)

valuation_summary_formatted = valuation_summary_formatted[[
    "Method",
    "Implied Equity Value (USD bn)",
    "Implied Share Price (USD)"
]]

valuation_summary_formatted

,Method,Implied Equity Value (USD bn),Implied Share Price (USD)
0,P/E,28.360000,11.040000
1,EV/EBITDA,107.060000,41.680000
2,EV/Operating CF,91.060000,35.450000
3,DCF,130.630000,50.840000


In [151]:
def esg_proxy_assessment(info):
    result = {}
  
    if info.get("sector") == "Energy":
        result["Environmental Risk"] = "High"
    else:
        result["Environmental Risk"] = "Medium"
 
    employees = info.get("fullTimeEmployees", 0)
    if employees > 50000:
        result["Social Risk"] = "Medium"
    else:
        result["Social Risk"] = "Low"
    
    roe = info.get("returnOnEquity", 0)
    debt_to_equity = info.get("debtToEquity", 0)
    
    if roe < 0.05 or debt_to_equity > 150:
        result["Governance Risk"] = "Weak"
    else:
        result["Governance Risk"] = "Acceptable"
    
    if result["Environmental Risk"] == "High":
        result["Overall ESG Risk"] = "Elevated"
    else:
        result["Overall ESG Risk"] = "Moderate"
    
    return result

bp_esg_proxy = esg_proxy_assessment(bp_info)
bp_esg_proxy

{'Environmental Risk': 'High',
 'Social Risk': 'Medium',
 'Governance Risk': 'Weak',
 'Overall ESG Risk': 'Elevated'}

In [152]:
risk_dashboard = pd.DataFrame({
    "Risk Dimension": [
        "Environmental (ESG)",
        "Social (ESG)",
        "Governance (ESG)",
        "Leverage",
        "Liquidity",
        "Cash Flow Stability",
        "Earnings Volatility"
    ],
    "Risk Level": [
        bp_esg_proxy["Environmental Risk"],
        bp_esg_proxy["Social Risk"],
        bp_esg_proxy["Governance Risk"],
        fundamental_risks["Leverage"],
        fundamental_risks["Liquidity"],
        fundamental_risks["Cash Flow Stability"],
        fundamental_risks["Earnings Volatility"]
    ]
})

risk_dashboard

,Risk Dimension,Risk Level
0,Environmental (ESG),High
1,Social (ESG),Medium
2,Governance (ESG),Weak
3,Leverage,High
4,Liquidity,Medium
5,Cash Flow Stability,Weak
6,Earnings Volatility,High


In [16]:
memo_input = {
    "company_name": "BP p.l.c. (ADR)",
    "ticker": "BP",
    "analysis_date": "End-2024",
    "analyst_view": "Bullish",

    "valuation_summary": {
        "dcf_intrinsic_price_usd": 50.72,
        "market_price_usd": 27.44,
        "dcf_upside_pct": 84.83,
        "final_recommendation": "BUY"
    },

    "financial_performance": {
        "profitability": {
            "roe_pct": {
                "2020": -0.24,
                "2021": 8.36,
                "2022": -3.00,
                "2023": 17.82,
                "2024": 0.49
            },
            "net_margin_pct": {
                "2020": -19.17,
                "2021": 4.80,
                "2022": -1.03,
                "2023": 7.31,
                "2024": 0.20
            },
            "ebitda_margin_pct": {
                "2020": -6.65,
                "2021": 20.63,
                "2022": 13.40,
                "2023": 20.88,
                "2024": 14.79
            }
        }
    },

    "cash_flow": {
        "free_cash_flow_usd": {
            "2020": -1.44e10,
            "2021": 1.27e10,
            "2022": 2.89e10,
            "2023": 1.78e10,
            "2024": 1.20e10
        },
        "fcf_margin_pct": {
            "2020": -0.14,
            "2021": 8.07,
            "2022": 11.96,
            "2023": 8.52,
            "2024": 6.34
        }
    },

    "leverage_and_coverage": {
        "debt_to_equity": {
            "2020": 2.13,
            "2021": 2.18,
            "2022": 2.47,
            "2023": 2.28,
            "2024": 2.60
        },
        "interest_coverage": {
            "2020": 10.16,
            "2021": -7.05,
            "2022": -6.87,
            "2023": -9.41,
            "2024": -2.48
        }
    },

    "valuation_multiples": {
        "pe_implied_price_usd": 11.31,
        "ev_ebitda_implied_price_usd": 41.69
    },

    "peer_comparison": {
        "rule": (
            "Peer comparison MUST be multiples-based. "
            "Use EV/EBITDA as primary; P/E and EV/Operating CF as secondary. "
            "Do NOT use ROE or margins as the primary peer comparison metric."
        ),
        "peers": ["BP", "Shell", "ExxonMobil"],
        "multiples_ttm": [
            {
                "company": "BP",
                "pe": 60.03,
                "ev_ebitda": 4.57,
                "ev_operating_cf": 5.38
            },
            {
                "company": "Shell",
                "pe": 14.28,
                "ev_ebitda": 5.12,
                "ev_operating_cf": 5.36
            },
            {
                "company": "ExxonMobil",
                "pe": 18.70,
                "ev_ebitda": 9.53,
                "ev_operating_cf": 11.42
            }
        ]
    },

    "methodology_note": (
        "All quantitative metrics above are derived from prior Python-based "
        "financial analysis, ratio computation, and valuation models executed "
        "earlier in this notebook. Due to external API rate limits, these results "
        "are treated as a fixed analytical snapshot. The language model is used "
        "solely for structuring, interpretation, and professional presentation."
    )
}

print("memo_input (BP) ready for IC memo generation")


memo_input (BP) ready for IC memo generation


In [17]:
import json

OLLAMA_HOST = "http://localhost:11434"
OLLAMA_MODEL = "llama3.1:8b"   

def ollama_chat(prompt: str, system: str = "", temperature: float = 0.2) -> str:
    url = f"{OLLAMA_HOST}/api/chat"
    payload = {
        "model": OLLAMA_MODEL,
        "messages": (
            ([{"role": "system", "content": system}] if system else [])
            + [{"role": "user", "content": prompt}]
        ),
        "options": {
            "temperature": temperature,
        },
        "stream": False
    }
    r = requests.post(url, json=payload, timeout=600)
    r.raise_for_status()
    data = r.json()
    return data["message"]["content"]

In [18]:
def build_ic_memo_prompt(memo_input: dict, analyst_view: str = "Neutral") -> str:
    """
    IC memo prompt with controlled visual hierarchy:
    - Title > Section headers > Subsection headers
    - Tables included
    - Consistent font sizing in Markdown render
    """
    return f"""
You are preparing an INTERNAL Investment Committee (IC) memo for an equity investment.

Analyst stance (tone only, not to change the conclusion): {analyst_view}

========================
STRICT FORMATTING RULES (NON-NEGOTIABLE)
========================
Use Markdown ONLY. Follow this hierarchy exactly:

1) '# ' → Document title (use ONCE only)
2) '## ' → Main section headers (ALL same level and size)
3) '### ' → Subsection headers (ALL same level and size)
4) Bold text (**) → Allowed ONLY for key figures or conclusions, NEVER for headers

Additional rules:
- Do NOT use '####' or deeper levels.
- Do NOT use bold text as a substitute for headers.
- Do NOT use HTML, colors, emojis, or special symbols.
- Tables MUST be standard Markdown tables (pipes and dashes only).
- Tables should appear immediately after the subsection they relate to.
- Do NOT repeat headings.

========================
CONTENT RULES
========================
1) Use ONLY the data provided in DATA INPUT.
2) If data is missing, explicitly write "Not available".
3) Discounted Cash Flow (DCF) is the PRIMARY valuation anchor.
   - Multiples (P/E, EV/EBITDA) are SECONDARY cross-checks.
   - P/E must NOT override the DCF-based conclusion.
4) The FINAL recommendation MUST match the quantitative recommendation
   provided in DATA INPUT.
5) Target length: approximately 900–1200 words (1–2 A4 pages).

========================
REQUIRED STRUCTURE (FOLLOW EXACTLY)
========================

# BP p.l.c. (ADR) – Equity Investment Memorandum

## Investment Thesis
- 3–5 concise bullet points.
- Explicitly state DCF intrinsic value, market price, and upside/downside.
- Make the implied recommendation unmistakably clear.

## Company & Business Overview
- Describe the business model, integration, and geographic scope.
- Keep this section factual and concise.

## Financial Performance & Key Ratios

### Profitability and Returns
- Discuss ROE, net margin, and EBITDA margin trends using specific numbers.

| Year | ROE (%) | Net Margin (%) | EBITDA Margin (%) |
|------|---------|----------------|-------------------|
| Use provided data only |

### Balance Sheet and Leverage
- Discuss leverage metrics and financial risk.

## Valuation

### Discounted Cash Flow (Primary)
- Present intrinsic value, market price, and upside/downside clearly.
- Explain why DCF is the anchor valuation.

### Multiples Cross-Check
- Present P/E and EV/EBITDA implications.
- Explicitly explain any divergence versus DCF without overturning it.

## Peer Comparison
- Compare BP against peers qualitatively and quantitatively where available.
- If peer metrics are missing, state "Not available".

## Management Quality & Capital Allocation
- Assess management discipline, capital allocation, and balance sheet strategy.
- Avoid speculation; base assessment on provided data.

## Catalysts
- List 3–6 plausible, non-speculative catalysts.
- Keep each catalyst to one concise line.

## Risks
- Cover fundamental, financial, and ESG risks.
- Tie risks directly to margins, leverage, and cash flow volatility.

## Investment Recommendation
- Clearly restate the final recommendation (BUY / HOLD / SELL).
- Summarize:
  - 3 key reasons supporting the recommendation
  - 3 key risks that could challenge the thesis

========================
DATA INPUT (TREAT AS GROUND TRUTH)
========================
{memo_input}
""".strip()

In [19]:
system_msg = "You are a disciplined senior buy-side equity analyst. Write like an investment professional."

ic_prompt = build_ic_memo_prompt(memo_input=memo_input)

ic_memo_md = ollama_chat(
    prompt=ic_prompt,
    system=system_msg,
    temperature=0.2
)

In [20]:
from IPython.display import display, Markdown

display(Markdown(ic_memo_md))

# BP p.l.c. (ADR) – Equity Investment Memorandum

## Investment Thesis
- BP's diversified energy portfolio provides a stable revenue stream.
- The company has made significant strides in reducing greenhouse gas emissions.
- Our DCF analysis indicates an intrinsic value of **$50.72**, implying an upside of **84.83%** from the current market price of $27.44.

## Company & Business Overview
BP is a multinational energy company with operations in over 70 countries. The company's business model is centered around oil and gas production, refining, and marketing. BP has made significant investments in renewable energy sources, including wind and solar power.

## Financial Performance & Key Ratios

### Profitability and Returns
| Year | ROE (%) | Net Margin (%) | EBITDA Margin (%) |
|------|---------|----------------|-------------------|
| 2020 | -0.24    | -19.17        | -6.65             |
| 2021 | 8.36     | 4.8           | 20.63             |
| 2022 | -3.0     | -1.03         | 13.4              |
| 2023 | 17.82    | 7.31          | 20.88             |
| 2024 | 0.49     | 0.2           | 14.79             |

BP's profitability and returns have shown significant improvement in recent years, driven by cost-cutting measures and increased efficiency.

### Balance Sheet and Leverage
BP's debt-to-equity ratio has been steadily increasing over the past few years, reaching **2.6** in 2024. The company's interest coverage ratio has also been declining, reaching **-2.48** in 2024. This suggests that BP may be facing some financial risk.

## Valuation

### Discounted Cash Flow (Primary)
Our DCF analysis indicates an intrinsic value of **$50.72**, which is higher than the current market price of $27.44. This implies an upside of **84.83%** from the current market price.

### Multiples Cross-Check
Using P/E as a cross-check, we get an implied price of **$11.31**, which is lower than our DCF-based intrinsic value. Using EV/EBITDA as another cross-check, we get an implied price of **$41.69**, which is also lower than our DCF-based intrinsic value.

## Peer Comparison
| Company | P/E | EV/EBITDA | EV/Operating CF |
|---------|-----|------------|-----------------|
| BP      | 60.03 | 4.57       | 5.38            |
| Shell   | 14.28 | 5.12       | 5.36            |
| ExxonMobil | 18.7  | 9.53       | 11.42           |

BP's P/E ratio is significantly higher than its peers, while its EV/EBITDA and EV/Operating CF ratios are lower.

## Management Quality & Capital Allocation
BP has a strong management team with a track record of making strategic investments in renewable energy sources. The company has also made significant efforts to reduce greenhouse gas emissions.

## Catalysts
1. Continued growth in renewable energy sources.
2. Improved profitability and returns.
3. Reduced greenhouse gas emissions.
4. Increased efficiency and cost-cutting measures.
5. Strategic acquisitions and partnerships.
6. Improved management discipline and capital allocation.

## Risks
- Financial risk due to increasing debt-to-equity ratio and declining interest coverage ratio.
- ESG risks due to BP's exposure to climate change and environmental degradation.
- Regulatory risks due to changing government policies and regulations.

## Investment Recommendation
Based on our analysis, we recommend a **BUY** rating for BP p.l.c. (ADR). The company's diversified energy portfolio, improving profitability and returns, and strong management team make it an attractive investment opportunity. However, investors should be aware of the financial and ESG risks associated with the company.

Three key reasons supporting the recommendation:

1. Diversified energy portfolio provides a stable revenue stream.
2. Improving profitability and returns driven by cost-cutting measures and increased efficiency.
3. Strong management team with a track record of making strategic investments in renewable energy sources.

Three key risks that could challenge the thesis:

1. Financial risk due to increasing debt-to-equity ratio and declining interest coverage ratio.
2. ESG risks due to BP's exposure to climate change and environmental degradation.
3. Regulatory risks due to changing government policies and regulations.

In [21]:
with open("IC_memo.md", "w", encoding="utf-8") as f:
    f.write(ic_memo_md)

from markdown import markdown

html = markdown(
    ic_memo_md,
    extensions=["tables", "fenced_code"]
)

with open("IC_memo.html", "w", encoding="utf-8") as f:
    f.write(html)

print("Saved: IC_memo.html")

Saved: IC_memo.html
